In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
from copy import deepcopy
from astropy.stats import sigma_clip
from scipy.stats import iqr
from chainconsumer import ChainConsumer

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

fig_width_pt = 240.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]

from astropy import coordinates as coords
import astropy.units as u
from scipy.signal import savgol_filter
from astroquery.sdss import SDSS
import pysynphot as S
from tools import *

In [2]:
data = pd.read_csv('K2_all_PS1_psc_ev_crowding_sdss.csv')
#ind = np.isfinite(data['ps1g_synSDSS'].values)
#data = data.iloc[ind]

diff = abs(data['gMeanPSFMag'].values - data['cgMeanPSFMag'].values)
ind = diff < .01
data = data.iloc[ind]

ind = ((abs(data['zMeanPSFMagErr'].values) < 0.01) & 
       (abs(data['gMeanPSFMagErr'].values) < 0.01) & 
       (abs(data['rMeanPSFMagErr'].values) < 0.01) & 
       (abs(data['iMeanPSFMagErr'].values) < 0.01))
data = data.iloc[ind]


zpsr = np.load('output/zeropoint_kep.npy')
extinctions = np.load('extinctions.npy')

In [3]:
def magerr_2_fluxerr(magerr,flux):
    ferr = magerr*np.log(10)/2.5*flux
    return ferr

def Comp_mag(d):
    """
    Calculate using the errors in flux space
    """
    gc = 10**(-2/5 *(d['gMeanPSFMag'].values-25))
    ge = magerr_2_fluxerr(d['gMeanPSFMagErr'].values,gc)
    rc = 10**(-2/5 *(d['rMeanPSFMag'].values-25))
    re = magerr_2_fluxerr(d['rMeanPSFMagErr'].values,rc)
    ic = 10**(-2/5 *(d['iMeanPSFMag'].values-25))
    ie = magerr_2_fluxerr(d['iMeanPSFMagErr'].values,ic)
    zc = 10**(-2/5 *(d['zMeanPSFMag'].values-25))
    ze = magerr_2_fluxerr(d['zMeanPSFMagErr'].values,zc)
    
    c1=0.25925186;c2=0.4979566;c3=0.17658766;c4=0.06823672;c5=0.03316101
    dfdg = (c1*gc+c2*rc+c3*ic+c4*zc)*c5/rc*(gc/rc)**(c5-1) + c1*(gc/rc)**c5
    dfdr = (c1*gc+c2*rc+c3*ic+c4*zc)*c5*gc/rc**2*(gc/rc)**(c5-1) + c2*(gc/rc)**c5
    dfdi = c3*(gc/rc)**5
    dfdz = c4*(gc/rc)**5
    
    err2 = dfdg**2*ge**2 + dfdr**2*re**2 + dfdi**2*ie**2 + dfdz**2*ze**2
    err = np.sqrt(err2)
    kcom = (c1*gc + c2*rc + c3*ic + c4*zc)*(gc/rc)**c5
    
    kmag = -2.5*np.log10(kcom) + 25
    kmagerr = 2.5/np.log(10) * (err/kcom)
    return np.array((kmag,kmagerr))


def magerr_2_fluxerr(magerr,flux):
    ferr = magerr*np.log(10)/2.5*flux
    return ferr

def Comp_mag2(d):
    """
    Calculate using the errors in magnitude space
    """
    gc = 10**(-2/5 *(d['gMeanPSFMag'].values-25))
    ge = d['gMeanPSFMagErr'].values
    rc = 10**(-2/5 *(d['rMeanPSFMag'].values-25))
    re = d['rMeanPSFMagErr'].values
    ic = 10**(-2/5 *(d['iMeanPSFMag'].values-25))
    ie = d['iMeanPSFMagErr'].values
    zc = 10**(-2/5 *(d['zMeanPSFMag'].values-25))
    ze = d['zMeanPSFMagErr'].values
    
    c1=0.25925186;c2=0.4979566;c3=0.17658766;c4=0.06823672;c5=0.03316101
    u = (c1*gc+c2*rc+c3*ic+c4*zc)
    
    dfdg = -2.5*(c1/u + c5/gc)
    dfdr = -2.5*(c2/u + c5/rc)
    dfdi = -2.5*c3/u
    dfdz = -2.5*c4/u
    
    err2 = dfdg**2*ge**2 + dfdr**2*re**2 + dfdi**2*ie**2 + dfdz**2*ze**2
    err = np.sqrt(err2)
    kcom = (c1*gc + c2*rc + c3*ic + c4*zc)*(gc/rc)**c5
    
    kmag = -2.5*np.log10(kcom) + 25
    kmagerr = err
    return np.array((kmag,kmagerr))
    